### Context

The number of restaurants in New York is increasing day by day. Lots of students and busy professionals rely on those restaurants due to their hectic lifestyles. Online food delivery service is a great option for them. It provides them with good food from their favorite restaurants. A food aggregator company FoodHub offers access to multiple restaurants through a single smartphone app.

The app allows the restaurants to receive a direct online order from a customer. The app assigns a delivery person from the company to pick up the order after it is confirmed by the restaurant. The delivery person then uses the map to reach the restaurant and waits for the food package. Once the food package is handed over to the delivery person, he/she confirms the pick-up in the app and travels to the customer's location to deliver the food. The delivery person confirms the drop-off in the app after delivering the food package to the customer. The customer can rate the order in the app. The food aggregator earns money by collecting a fixed margin of the delivery order from the restaurants.

### Objective

The food aggregator company has stored the data of the different orders made by the registered customers in their online portal. They want to analyze the data to get a fair idea about the demand of different restaurants which will help them in enhancing their customer experience. Suppose you are hired as a Data Scientist in this company and the Data Science team has shared some of the key questions that need to be answered. Perform the data analysis to find answers to these questions that will help the company to improve the business.

### Data Description

The data contains the different data related to a food order. The detailed data dictionary is given below.

### Data Dictionary

* order_id: Unique ID of the order
* customer_id: ID of the customer who ordered the food
* restaurant_name: Name of the restaurant
* cuisine_type: Cuisine ordered by the customer
* cost: Cost of the order
* day_of_the_week: Indicates whether the order is placed on a weekday or weekend (The weekday is from Monday to Friday and the weekend is Saturday and Sunday)
* rating: Rating given by the customer out of 5
* food_preparation_time: Time (in minutes) taken by the restaurant to prepare the food. This is calculated by taking the difference between the timestamps of the restaurant's order confirmation and the delivery person's pick-up confirmation.
* delivery_time: Time (in minutes) taken by the delivery person to deliver the food package. This is calculated by taking the difference between the timestamps of the delivery person's pick-up confirmation and drop-off information

### Importing the required libraries

In [ ]:
# import libraries for data manipulation
import numpy as np
import pandas as pd

# import libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read the data
data = pd.read_csv('data/foodhub_order.csv')
# Copying data to another variable to avoid any changes to the original data
df = data.copy()
# returns the first 5 rows
df.head()

In [ ]:
# Printing number of count of each unique value in customer_id
print(df['customer_id'].value_counts())
print('-' * 50)

#### Observations:

The DataFrame has 9 columns as mentioned in the Data Dictionary. Data in each row corresponds to the order placed by a customer.

The rating has a non numerical value: 'Not given'.

The column day_of_the_week is a categorical variable.

The column customer_id in this context represents a categorical variable.
The above output shows which of the customer could be a potencial loyal client of the application.

The column order_id doesn't add valuable information for the problem context

In [ ]:
df.shape

The dataframe contains 1898 rows and 9 columns

In [ ]:
df.info()

#### Observations:

* All columns have 1898 observations. This indicates that there aren't 'Null' values presents.
* The columns order_id, customer_id, food_preparation_time and delivery time are integers data types.
* The columns restaurant_name, cuisine_type, day_of_the_week and rating are object data type.
* The column cost_of_the_order is a float data type.

In [ ]:
df.isnull().sum()

* There are no missing values in the data.

In [ ]:
# Since order_id doesn't add valuable information it will be droped
df = df.drop('order_id', axis=1)
#Calculate the statistical summary for numerical columns
df.describe().T

In [ ]:
df.describe(exclude = 'number').T

#### Observations:


*   The minimun time that takes for food to be prepared is 20 min
*   The average time that takes for food to be prepared is 27 min
*   The maximun time that takes for food to be prepared is 35 min

In [ ]:
order_no_rating = df[df['rating'] == 'Not given']
print(order_no_rating['rating'].value_counts())
print('-' * 50)

#### Observations:
There are 736 order not rated from a total of 1898 orders. That is a 38.8% of the orders in the sample are not rated.

### Exploratory Data Analysis (EDA)

### Univariate Analysis

In [ ]:
num_cols = ['cost_of_the_order', 'food_preparation_time', 'delivery_time']
cat_cols = ['customer_id', 'restaurant_name', 'cuisine_type', 'day_of_the_week', 'rating']

def histogram_boxplot(feature, figsize=(15, 10), bins="auto"):
    """ Boxplot and histogram combined
    feature: 1-d feature array
    figsize: size of fig (default (15, 10))
    bins: number of bins (default "auto")
    """
    f, (ax_box, ax_hist) = plt.subplots( # Crea un grid de figuras
        nrows=2,  # Number of rows of the subplot grid
        sharex=True,  # The X-axis will be shared among all the subplots
        gridspec_kw={"height_ratios": (.25, .75)}, # Distribucion de las figuras (25% y 75%)
        figsize=figsize
    )

    # Creating the subplots
    # Boxplot will be created and the mean value of the column will be indicated using some symbol
    sns.boxplot(x=feature, ax=ax_box, showmeans=True, color='red')

    # For histogram
    sns.histplot(x=feature, kde=True, ax=ax_hist, bins=bins)
    ax_hist.axvline(np.mean(feature), color='g', linestyle='--')      # Add mean to the histogram
    ax_hist.axvline(np.median(feature), color='black', linestyle='-') # Add median to the histogram

    plt.show()

#### **Observations on cost of orders**

In [ ]:
histogram_boxplot(df.cost_of_the_order)

**Observations:**
* The distribution of cost_of_order is little left-skewed.
* Median cost_of_the_order is  ~14.14, but the mean is ~16.
* There are not outliers in this variable.
* The most cost_of_order are at the lower end, between 12 and the mean (~16).

#### **Observations on delivery_time**

In [ ]:
histogram_boxplot(df.delivery_time)

**Observations:**
* The distribution of delivery_time is a bit right skewed.
* The peaks could be related to the type of cuisine.
* Median= 25 and the mean= 24 are close.
* There are not outliers in this variable.

#### **Let's explore the categorical variables**

In [ ]:
def bar_perc(data, z):
    total = len(data[z]) # Length of the column
    plt.figure(figsize = (15, 5))

    # Convert the column to a categorical data type
    data[z] = data[z].astype('category')

    ax = sns.countplot(x=z, data=data, palette='Paired', order=data[z].value_counts().index)

    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height() / total) # Percentage of each class
        x = p.get_x() + p.get_width() / 2 - 0.05                    # Width of the plot
        y = p.get_y() + p.get_height()                              # Height of the plot
        ax.annotate(percentage, (x, y), size = 12)                  # Annotate the percentage

    plt.show()

#### **Observations on Cuisine Type**

In [ ]:
bar_perc(df, 'cuisine_type')

**Observation:**

* The four most popular cuisine types are the American, Japanese, Italian and Chinese, concentrating around 80% of the orders.
* The others cuisine type (10 types) are less frequently ordered.

#### **Observations on Day of the Week**

In [ ]:
bar_perc(df, 'day_of_the_week')

**Observation:**

* The most of the order are made on weekends, concentrating around 71% of the orders.

#### **Observations on Rating**

In [ ]:
bar_perc(df, 'rating')

**Observation:**

* A significant portion of orders —38.8%— were not rated by customers.
* The restaurants with a positive rating (4-5) represents the 51% of the orders.

 **Top 5 restaurants in terms of the number of orders received**

In [ ]:
df['restaurant_name'].value_counts().head()

#### Observations:

The top 5 restaurants are:


1.   Shake Shack	               with 219 orders
2.   The Meatball Shop	         with 132 orders
3.   Blue Ribbon Sushi	         with 119 orders
4.   Blue Ribbon Fried Chicken	 with 96 orders
5.   Parm	                       with 68 orders


**The most popular cuisine on weekends**

In [ ]:
filter_weekend = df['day_of_the_week'] == 'Weekend'
df[filter_weekend]['cuisine_type'].value_counts().head(1)

#### Observations:
The most popular cuisine on weekends is the American cuisine.

**Percentage of the orders cost more than 20 dollars**

In [ ]:
filter_cost = df['cost_of_the_order'] > 20
df[filter_cost].shape[0] / df.shape[0] * 100

#### Observations:
The percentage of the orders cost more than 20 dollars is 29,24%

**The mean order delivery time**

In [ ]:
df['delivery_time'].mean()

#### Observations:
The mean order delivery time is 24 min

**The top 3 most frequent customers**

In [ ]:
#Top 3 most frequent customer
top_customer = df['customer_id'].value_counts().head(3)
print(top_customer)
print('-' * 50)


#### Observations:
The top 3 most frequent customers are:
1.   Customer ID 52832   with  13 orders.
2.   Customer ID 47440   with 10 orders.
3.   Customer ID 83287   with  9 orders.
